# Sistema para recomendação de livros

In [13]:
# Instalar a biblioteca surprise que será utilizada no modelo SVD
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163005 sha256=35050cc91d62a2e3d1370f56c089686f1ce57fd268c9e2cecd285566fa795f00
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [1]:
# Importar os pacotes necessários
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy


In [2]:
# Carregar o arquivo de livros e visualizar primeiras linhas
livros = pd.read_csv("BX_Books.csv", sep=";", encoding="latin1")
livros.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [3]:
livros.shape

(271379, 8)

In [4]:
# Carregar o arquivo de avaliações e visualizar primeiras linhas
avaliacoes = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding="latin1")
avaliacoes.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [5]:
avaliacoes.shape

(1149780, 3)

## Pré-processamento dos Dados

In [6]:
# Contar o número de avaliações por livro
contagem_avaliacoes = avaliacoes.groupby('ISBN').size().reset_index(name='QTDE_AVALIACOES')

# Filtrar os livros com 10 ou mais avaliações
livros_filtrados = contagem_avaliacoes[contagem_avaliacoes['QTDE_AVALIACOES'] >= 10]

# Filtrar os livros avaliados
livros_avaliados = livros.merge(livros_filtrados, on='ISBN', how='inner')

# Filtrar as avaliações para incluir apenas os livros avaliados
avaliacoes = avaliacoes[avaliacoes['ISBN'].isin(livros_avaliados['ISBN'])]

# Selecionar apenas as variáveis necessárias dos livros
livros_avaliados = livros_avaliados[['ISBN', 'Book-Title']]

# Renomear as colunas
livros_avaliados.rename(columns={'ISBN': 'ID_LIVRO', 'Book-Title': 'TITULO'}, inplace=True)

<ipython-input-6-6359f2131773>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  livros_avaliados.rename(columns={'ISBN': 'ID_LIVRO', 'Book-Title': 'TITULO'}, inplace=True)


In [7]:
# Visualizar as primeiras linhas
avaliacoes.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
2,276727,0446520802,0
8,276744,038550120X,7
10,276746,0425115801,0
11,276746,0449006522,0


In [8]:
# Visualizar primeiras linhas
livros_avaliados.head(5)

,ID_LIVRO,TITULO
0,0002005018,Clara Callan
1,0374157065,Flu: The Story of the Great Influenza Pandemic...
2,0399135782,The Kitchen God's Wife
3,0440234743,The Testament
4,0452264464,Beloved (Plume Contemporary Fiction)


In [9]:
# Pré-processar os títulos dos livros
livros_avaliados['TITULO'] = livros_avaliados['TITULO'].str.lower()
livros_avaliados['TITULO'] = livros_avaliados['TITULO'].str.replace(r'\W', ' ')
livros_avaliados['TITULO'] = livros_avaliados['TITULO'].str.replace(r'\s+', ' ')

In [10]:
# Visualizar as primeiras linhas
livros_avaliados.head(5)

,ID_LIVRO,TITULO
0,0002005018,clara callan
1,0374157065,flu: the story of the great influenza pandemic...
2,0399135782,the kitchen god's wife
3,0440234743,the testament
4,0452264464,beloved (plume contemporary fiction)


Preparar os dados para utilizrmos o modelo SVD (Decomposição de Valor Singular), comumente utilizado para sistemas de recomendação



In [12]:
# Dividir os dados em conjunto de treinamento e teste para SVD
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(avaliacoes[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = surprise_train_test_split(data, test_size=0.2, random_state=42)

# Inicializar e treinar o modelo SVD
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Fazer previsões nos dados de teste usando SVD
predictions_svd = modelo_svd.test(testset)

Calcular as métricas do modelo SVD

In [13]:
# Avaliar o desempenho do modelo SVD
rmse_svd = accuracy.rmse(predictions_svd)
mae_svd = accuracy.mae(predictions_svd)

print("Desempenho do modelo SVD:")
print("RMSE:", rmse_svd)
print("MAE:", mae_svd)

RMSE: 3.6311
MAE:  3.0781
Desempenho do modelo SVD:
RMSE: 3.6311292003055464
MAE: 3.0780561117345826


Preparar os dados para rodar o modelo KNN, também utilizado para sistemas de recomendação, quando o conjunto de dados não é tão grande

In [14]:
# Inicializar e treinar o modelo KNN
vectorizer = TfidfVectorizer(stop_words='english')
titulo_tfidf = vectorizer.fit_transform(livros_avaliados['TITULO'])

# Dividir os dados em treinamento e teste
X_train, X_test = train_test_split(titulo_tfidf, test_size=0.2, random_state=42)

knn = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine')
knn.fit(X_train)

# Fazer previsões nos dados de teste usando KNN
distances, indices = knn.kneighbors(X_test)

Calcular as métricas do modelo KNN

In [15]:
# Calcular as métricas de desempenho do modelo KNN
rmse_knn = np.sqrt(np.mean(np.square(distances)))  # RMSE é a raiz quadrada da média dos quadrados das distâncias
mae_knn = np.mean(distances)  # MAE é a média das distâncias

print("\nDesempenho do modelo KNN:")
print("RMSE:", rmse_knn)
print("MAE:", mae_knn)


Desempenho do modelo KNN:
RMSE: 0.5670021171155796
MAE: 0.5056771296156916
